# How to use Different models with OpenAI Agents SDK that return Structured Output


In [1]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print("OpenAI API key is set")
else:
    print("OpenAI API key is not set")

if gemini_api_key:
    print("Gemini API key is set")
else:
    print("Gemini API key is not set")
    
if groq_api_key:
    print("Groq API key is set")
else:
    print("Groq API key is not set")

OpenAI API key is set
Gemini API key is set
Groq API key is set


In [4]:
gemini_base_url = os.getenv('GEMINI_BASE_URL')
groq_base_url = os.getenv('GROQ_BASE_URL')

if gemini_base_url:
    print("Gemini API base URL is set")
else:
    print("Gemini API base URL is not set")

if groq_base_url:
    print("Groq API base URL is set")
else:
    print("Groq API base URL is not set")

Gemini API base URL is set
Groq API base URL is set


# Configure any models with OpenAI Agents SDK

In [5]:
openai_client = AsyncOpenAI() # default base url is https://api.openai.com/v1 with OpenAI Agents SDK
gemini_client = AsyncOpenAI(api_key=gemini_api_key, base_url=gemini_base_url)
groq_client = AsyncOpenAI(api_key=groq_api_key, base_url=groq_base_url)

print(openai_client)
print(gemini_client)
print(groq_client)

In [6]:
# openai_model_name = "gpt-4o-mini"
gemini_model_name = "gemini-2.0-flash"
# groq_model_name = "llama-3.3-70b-versatile"

# print(openai_model_name)
print(gemini_model_name)
# print(groq_model_name)

gemini-2.0-flash


In [7]:
# openai_model = OpenAIChatCompletionsModel(openai_client=openai_client, model=openai_model_name)
gemini_model = OpenAIChatCompletionsModel(openai_client=gemini_client, model=gemini_model_name)
# groq_model = OpenAIChatCompletionsModel(openai_client=groq_client, model=groq_model_name)


# print(openai_model)
print(gemini_model)
# print(groq_model)

# Structured Output
1. Create a Pydantic Model
2. pass it as `output_type` in Agent Object

In [8]:
class ResponseStructure(BaseModel):
    who_am_i_testing: str

In [9]:
# test_openai_model_agent = Agent(name="test_openai_model_agent", instructions="You are a test agent that uses the OpenAI model to generate a response.", model=openai_model, output_type=ResponseStructure)
test_gemini_model_agent = Agent(name="test_gemini_model_agent", instructions="You are a test agent that uses the Gemini model to generate a response.", model=gemini_model, output_type=ResponseStructure)
# Seems like Groq doesn't support output_type yet, we need to parse it manually using the prompt
# test_groq_model_agent = Agent(name="test_groq_model_agent", instructions="You are a test agent that uses the Llama-3.3 model provided through Groq to generate a response.", model=groq_model) 

# print(test_openai_model_agent)
print(test_gemini_model_agent)
# print(test_groq_model_agent)

Agent(name='test_gemini_model_agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a test agent that uses the Gemini model to generate a response.', prompt=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x1152cb800>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=<class '__main__.ResponseStructure'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)


In [10]:
user_prompt = "I am testing you."

# groq_user_prompt_as_it_does_not_support_output_type = f"""
# You are a test agent that uses the model provided by Groq to generate a response.
# The user prompt is: {user_prompt}

# Please response in the following JSON format:
# {{
#     "who_am_i_testing": string
# }}
# """

with trace("Multi Model Integration with Structured Output") as t:
    # openai_agent_result = await Runner.run(test_openai_model_agent, user_prompt)

    gemini_agent_result = await Runner.run(test_gemini_model_agent, user_prompt)

    # groq_agent_result = await Runner.run(test_groq_model_agent, groq_user_prompt_as_it_does_not_support_output_type)

# print(openai_agent_result.final_output)
print(gemini_agent_result.final_output)
# print(groq_agent_result.final_output)

who_am_i_testing='Gemini'
